[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/kata_04_temple_bell_ceremony_classifier_unrevised.ipynb)

## 🏮 The Ancient Scroll Unfurls 🏮

**THE SACRED BELLS OF CEREMONY WISDOM**

Dan Level: 1 (Temple Sweeper) | Time: 45 minutes | Sacred Arts: Multi-class Classification, Softmax Activation, Cross-Entropy Loss

## 📜 THE CHALLENGE

Master Pai-Torch sits in the temple's bell tower, surrounded by four ancient bronze bells that have rung for centuries. "These sacred bells have marked the rhythm of our temple life for generations, Grasshopper. Each ceremony—morning meditation, noon contemplation, evening reflection, and midnight transcendence—requires its own unique tone." The master's weathered hands gesture toward the bells, each showing the patina of countless ringings. "But time and weather have changed their voices, and even the most experienced monks sometimes struggle to identify which ceremony should begin when they hear a bell's call."

"Your task is to restore this lost wisdom," Master Pai-Torch continues, eyes twinkling with the promise of hidden knowledge. "Using only the pitch frequency of a bell's tone, you must create a neural oracle that can distinguish between all four ceremonies. This will teach you the art of multi-class classification—how a single feature can reveal multiple truths, and how the softmax function transforms raw predictions into the language of probability and decision."

## 🎯 THE SACRED OBJECTIVES

By completing this kata, you will:
- [ ] Master multi-class classification with PyTorch
- [ ] Understand softmax activation for probability distributions
- [ ] Implement cross-entropy loss for multiple classes
- [ ] Learn to interpret class probabilities and make confident predictions
- [ ] Practice single-variable feature classification
- [ ] Understand the difference between binary and multi-class classification

In [ ]:
# 📦 FIRST CELL - ALL IMPORTS AND CONFIGURATION
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple
import torch.nn.functional as F

# Set reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Global configuration constants
DEFAULT_CHAOS_LEVEL = 0.1
SACRED_SEED = 42
N_CEREMONIES = 4
CEREMONY_NAMES = ['Morning Meditation', 'Noon Contemplation', 'Evening Reflection', 'Midnight Transcendence']
CEREMONY_COLORS = ['gold', 'orange', 'purple', 'darkblue']

## 🔔 THE SACRED DATA GENERATION SCROLL

In [ ]:
def generate_bell_ceremony_data(n_observations: int = 400, chaos_level: float = 0.1,
                               sacred_seed: int = 42) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate observations of temple bell frequencies and their corresponding ceremonies.
    
    Ancient bell master wisdom:
    - Morning Meditation: 220-280 Hz (bright, awakening tones)
    - Noon Contemplation: 320-380 Hz (focused, clear tones) 
    - Evening Reflection: 180-240 Hz (warm, settling tones)
    - Midnight Transcendence: 400-460 Hz (ethereal, transcendent tones)
    
    Args:
        n_observations: Number of bell ringings to simulate (100 per ceremony)
        chaos_level: Amount of bell aging and weather effects (0.0 = perfect bells, 1.0 = ancient chaos)
        sacred_seed: Ensures consistent randomness for reproducible bell wisdom
        
    Returns:
        Tuple of (frequencies, ceremony_labels) as sacred tensors
        - frequencies: Bell pitch frequencies in Hz
        - ceremony_labels: Integer labels (0=Morning, 1=Noon, 2=Evening, 3=Midnight)
    """
    torch.manual_seed(sacred_seed)
    np.random.seed(sacred_seed)
    
    # Each ceremony gets equal representation
    per_ceremony = n_observations // N_CEREMONIES
    
    # The sacred frequency ranges known to bell masters
    frequency_ranges = {
        0: (220, 280),  # Morning Meditation - bright awakening
        1: (320, 380),  # Noon Contemplation - focused clarity
        2: (180, 240),  # Evening Reflection - warm settling
        3: (400, 460)   # Midnight Transcendence - ethereal heights
    }
    
    all_frequencies = []
    all_labels = []
    
    for ceremony_id in range(N_CEREMONIES):
        min_freq, max_freq = frequency_ranges[ceremony_id]
        
        # Generate base frequencies for this ceremony
        base_frequencies = torch.rand(per_ceremony) * (max_freq - min_freq) + min_freq
        
        # Add chaos from years of temple weather and aging bronze
        chaos = torch.randn(per_ceremony) * chaos_level * (max_freq - min_freq) * 0.3
        ceremonial_frequencies = base_frequencies + chaos
        
        # Even ancient bells have physical limits
        ceremonial_frequencies = torch.clamp(ceremonial_frequencies, 100, 600)
        
        all_frequencies.append(ceremonial_frequencies)
        all_labels.extend([ceremony_id] * per_ceremony)
    
    # Combine all ceremonial data
    frequencies = torch.cat(all_frequencies)
    ceremony_labels = torch.tensor(all_labels, dtype=torch.long)
    
    # Shuffle the data like bells ringing throughout the day
    shuffle_indices = torch.randperm(len(frequencies))
    frequencies = frequencies[shuffle_indices].unsqueeze(1)
    ceremony_labels = ceremony_labels[shuffle_indices]
    
    return frequencies, ceremony_labels

def visualize_bell_wisdom(frequencies: torch.Tensor, labels: torch.Tensor,
                         predictions: torch.Tensor = None, probabilities: torch.Tensor = None):
    """Display the sacred patterns of temple bell ceremonies."""
    plt.figure(figsize=(15, 10))
    
    # Create subplot layout
    if predictions is not None:
        plt.subplot(2, 2, (1, 2))  # Top row spans both columns
    
    # Main scatter plot of bell frequencies by ceremony
    for ceremony_id in range(N_CEREMONIES):
        mask = labels == ceremony_id
        ceremony_freqs = frequencies[mask]
        
        # Add small random y-offset for visibility
        y_positions = torch.full((len(ceremony_freqs),), ceremony_id + 0.4) + \
                     torch.randn(len(ceremony_freqs)) * 0.15
        
        plt.scatter(ceremony_freqs.numpy(), y_positions.numpy(), 
                   alpha=0.6, color=CEREMONY_COLORS[ceremony_id], s=40,
                   label=f'{CEREMONY_NAMES[ceremony_id]}')
    
    plt.xlabel('Bell Frequency (Hz)')
    plt.ylabel('Ceremony Type')
    plt.title('The Sacred Frequencies of Temple Bell Ceremonies', fontsize=14, fontweight='bold')
    plt.yticks(range(N_CEREMONIES), CEREMONY_NAMES)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    if predictions is not None:
        # Prediction accuracy subplot
        plt.subplot(2, 2, 3)
        correct = (predictions == labels).float()
        accuracy = correct.mean().item()
        
        # Accuracy by frequency range
        freq_bins = torch.linspace(frequencies.min(), frequencies.max(), 20)
        bin_accuracies = []
        bin_centers = []
        
        for i in range(len(freq_bins) - 1):
            mask = (frequencies.squeeze() >= freq_bins[i]) & (frequencies.squeeze() < freq_bins[i+1])
            if mask.sum() > 0:
                bin_acc = correct[mask].mean().item()
                bin_accuracies.append(bin_acc)
                bin_centers.append((freq_bins[i] + freq_bins[i+1]) / 2)
        
        plt.plot(bin_centers, bin_accuracies, 'o-', color='gold', linewidth=2, markersize=6)
        plt.axhline(y=accuracy, color='red', linestyle='--', alpha=0.7, 
                   label=f'Overall Accuracy: {accuracy:.1%}')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Prediction Accuracy')
        plt.title('Wisdom Accuracy Across Frequencies')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.ylim(0, 1.1)
        
        # Confusion matrix visualization
        plt.subplot(2, 2, 4)
        confusion = torch.zeros(N_CEREMONIES, N_CEREMONIES)
        for true_label in range(N_CEREMONIES):
            for pred_label in range(N_CEREMONIES):
                mask = (labels == true_label) & (predictions == pred_label)
                confusion[true_label, pred_label] = mask.sum().item()
        
        # Normalize by true labels
        confusion_norm = confusion / confusion.sum(dim=1, keepdim=True)
        
        im = plt.imshow(confusion_norm.numpy(), cmap='YlOrRd', vmin=0, vmax=1)
        plt.colorbar(im, fraction=0.046, pad=0.04)
        
        # Add text annotations
        for i in range(N_CEREMONIES):
            for j in range(N_CEREMONIES):
                text = f'{confusion_norm[i, j]:.2f}'
                plt.text(j, i, text, ha='center', va='center', 
                        color='white' if confusion_norm[i, j] > 0.5 else 'black')
        
        plt.xlabel('Predicted Ceremony')
        plt.ylabel('True Ceremony')
        plt.title('Bell Wisdom Confusion Matrix')
        plt.xticks(range(N_CEREMONIES), ['Morning', 'Noon', 'Evening', 'Midnight'], rotation=45)
        plt.yticks(range(N_CEREMONIES), ['Morning', 'Noon', 'Evening', 'Midnight'])
    
    plt.tight_layout()
    plt.show()

def demonstrate_bell_mystery():
    """Show how different frequencies map to different ceremonies."""
    print("🔔 Master Pai-Torch demonstrates the Bell Mystery:")
    print("\n   ✨ Each ceremony has its sacred frequency range:")
    print("      • Morning Meditation: 220-280 Hz (bright awakening)")
    print("      • Noon Contemplation: 320-380 Hz (focused clarity)")
    print("      • Evening Reflection: 180-240 Hz (warm settling)")
    print("      • Midnight Transcendence: 400-460 Hz (ethereal heights)")
    print("\n   🎯 Your neural oracle must learn these ancient patterns!")

# Generate the sacred data
frequencies, ceremony_labels = generate_bell_ceremony_data(n_observations=400)

print(f"📊 Generated {len(frequencies)} bell observations")
print(f"   Frequency range: {frequencies.min():.1f} - {frequencies.max():.1f} Hz")
print(f"   Ceremonies represented: {len(torch.unique(ceremony_labels))}")

demonstrate_bell_mystery()
visualize_bell_wisdom(frequencies, ceremony_labels)

## 🧘 FIRST MOVEMENTS: THE BELL CEREMONY CLASSIFIER

In [ ]:
class BellCeremonyOracle(nn.Module):
    """A mystical artifact for understanding temple bell ceremony patterns."""
    
    def __init__(self, input_features: int = 1, n_ceremonies: int = 4):
        super(BellCeremonyOracle, self).__init__()
        # TODO: Create the Linear layer for multi-class classification
        # Hint: Input is frequency (1 feature), output is 4 ceremony classes
        # Remember: No activation here - we'll use softmax later!
        self.linear = None
        
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """Channel bell frequencies through the mystical network."""
        # TODO: Pass the input through your Linear layer
        # The result will be "logits" - raw scores for each ceremony
        logits = None
        return logits
    
    def predict_ceremony(self, features: torch.Tensor) -> torch.Tensor:
        """Predict the most likely ceremony for given frequencies."""
        with torch.no_grad():
            logits = self.forward(features)
            # Apply softmax to get probabilities
            probabilities = F.softmax(logits, dim=1)
            # Get the ceremony with highest probability
            predictions = torch.argmax(probabilities, dim=1)
            return predictions, probabilities

def train_ceremony_oracle(model: nn.Module, features: torch.Tensor, target: torch.Tensor,
                         epochs: int = 1000, learning_rate: float = 0.01) -> list:
    """
    Train the bell ceremony classification oracle.
    
    Returns:
        List of loss values during training
    """
    # TODO: Choose your loss calculation method for multi-class classification
    # Hint: CrossEntropyLoss is the sacred choice for multiple classes
    # It combines softmax + negative log likelihood automatically!
    criterion = None
    
    # TODO: Choose your parameter updating method
    # Hint: SGD remains the traditional path for this level
    optimizer = None
    
    losses = []
    accuracies = []
    
    for epoch in range(epochs):
        # TODO: CRITICAL - Clear the gradient spirits from previous cycle
        # Hint: The spirits accumulate if not banished properly
        
        # TODO: Forward pass - get ceremony predictions (logits)
        logits = None
        
        # TODO: Compute the loss
        # Note: CrossEntropyLoss expects logits (raw scores) and target labels
        loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update parameters
        
        # Calculate accuracy for monitoring
        with torch.no_grad():
            predicted_ceremonies = torch.argmax(F.softmax(logits, dim=1), dim=1)
            accuracy = (predicted_ceremonies == target).float().mean().item()
            accuracies.append(accuracy)
        
        losses.append(loss.item())
        
        # Report progress to Master Pai-Torch
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.1%}')
            if accuracy > 0.8:
                print("🔔 The Bell Spirits harmonize with your understanding!")
    
    return losses, accuracies

# Create the sacred oracle
model = BellCeremonyOracle()
print("🏮 Bell Ceremony Oracle created!")
print(f"   Architecture: {model}")
print("\n💫 Ready for training when you complete the TODOs above...")

## ⚡ THE TRIALS OF MASTERY

### Trial 1: Basic Multi-Class Mastery
- [ ] Loss decreases consistently (no chaotic Bell Spirits)
- [ ] Final accuracy above 75% (Master Pai-Torch's minimum wisdom threshold)
- [ ] Model learns distinct ceremony patterns
- [ ] Confusion matrix shows clear diagonal pattern

### Trial 2: Understanding Test

In [ ]:
def test_your_wisdom(model):
    """Master Pai-Torch's evaluation of your multi-class understanding."""
    # Test with representative frequencies from each ceremony
    test_frequencies = torch.tensor([[250.0],  # Morning
                                   [350.0],  # Noon  
                                   [200.0],  # Evening
                                   [430.0]]) # Midnight
    
    predictions, probabilities = model.predict_ceremony(test_frequencies)
    
    # Check output shapes
    assert predictions.shape == (4,), "Predictions must have shape (4,)!"
    assert probabilities.shape == (4, 4), "Probabilities must have shape (4, 4)!"
    
    # Check that probabilities sum to 1 for each prediction
    prob_sums = probabilities.sum(dim=1)
    assert torch.allclose(prob_sums, torch.ones(4), atol=1e-6), "Probabilities must sum to 1!"
    
    # Test frequency understanding
    expected_ceremonies = [0, 1, 2, 3]  # Morning, Noon, Evening, Midnight
    correct_predictions = (predictions == torch.tensor(expected_ceremonies)).sum().item()
    
    print("🔔 Bell Frequency Test Results:")
    for i, (freq, pred, expected) in enumerate(zip(test_frequencies, predictions, expected_ceremonies)):
        prob = probabilities[i, pred].item()
        ceremony_name = CEREMONY_NAMES[pred]
        expected_name = CEREMONY_NAMES[expected]
        correct = "✅" if pred == expected else "❌"
        print(f"   {freq.item():.0f} Hz → {ceremony_name} ({prob:.1%} confidence) {correct}")
        print(f"      Expected: {expected_name}")
    
    accuracy = correct_predictions / 4
    if accuracy >= 0.75:
        print(f"\n🎉 Master Pai-Torch nods with approval - {accuracy:.0%} understanding achieved!")
        print("   'The bells sing in harmony with your neural wisdom, young one.'")
    else:
        print(f"\n🤔 Master Pai-Torch strokes beard thoughtfully - {accuracy:.0%} understanding")
        print("   'The bells whisper secrets your network has not yet learned to hear.'")
    
    return accuracy

def analyze_ceremony_confidence(model, frequencies, labels):
    """Analyze how confident the model is in its ceremony predictions."""
    predictions, probabilities = model.predict_ceremony(frequencies)
    
    # Get confidence (max probability) for each prediction
    confidences = torch.max(probabilities, dim=1)[0]
    correct = (predictions == labels).float()
    
    print("🎯 Ceremony Prediction Analysis:")
    print(f"   Average confidence: {confidences.mean():.1%}")
    print(f"   Confidence when correct: {confidences[correct == 1].mean():.1%}")
    print(f"   Confidence when wrong: {confidences[correct == 0].mean():.1%}")
    
    # Analyze per ceremony
    print("\n📊 Per-Ceremony Analysis:")
    for ceremony_id in range(N_CEREMONIES):
        mask = labels == ceremony_id
        ceremony_correct = correct[mask]
        ceremony_conf = confidences[mask]
        
        accuracy = ceremony_correct.mean().item()
        avg_conf = ceremony_conf.mean().item()
        
        print(f"   {CEREMONY_NAMES[ceremony_id]}: {accuracy:.1%} accuracy, {avg_conf:.1%} confidence")

# Placeholder for after training
print("🧪 Wisdom tests ready - complete training first to unlock!")

## 🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

### Extension 1: Cook Oh-Pai-Timizer's Frequency Fusion
*"Just as I blend spices to create new flavors, you can blend frequencies to enhance wisdom!"*

*Cook Oh-Pai-Timizer approaches with flour-dusted hands and a knowing smile*

*"Ah, grasshopper! You've learned to classify single bell tones—excellent! But in my kitchen, the best dishes combine multiple ingredients. What if we could make your oracle even wiser by adding a second feature? The bells don't just have pitch—they also have intensity (volume). Two ingredients, richer flavor!"*

**NEW CONCEPTS:** Multi-feature classification, feature combination, input dimensionality  
**DIFFICULTY:** +15% (still Dan 1, but with 2D input)

In [ ]:
def generate_enhanced_bell_data(n_observations: int = 400):
    """
    Generate bell data with both frequency AND intensity features.
    
    Ancient wisdom: Different ceremonies use different volumes:
    - Morning: Medium volume (awakening but not jarring)
    - Noon: High volume (calling all to contemplation)
    - Evening: Low volume (peaceful reflection)
    - Midnight: Very low volume (disturbing none)
    
    Returns:
        Tuple of (combined_features, ceremony_labels)
        combined_features shape: (n_observations, 2) [frequency, intensity]
    """
    # Get original frequency data
    frequencies, labels = generate_bell_ceremony_data(n_observations)
    
    # TODO: Generate intensity data based on ceremony type
    # Hint: Use ceremony labels to determine appropriate intensity ranges
    # Hint: Combine features using torch.cat along dimension 1
    pass

class EnhancedBellOracle(nn.Module):
    """A more sophisticated oracle that considers both frequency and intensity."""
    
    def __init__(self, input_features: int = 2, n_ceremonies: int = 4):
        super(EnhancedBellOracle, self).__init__()
        # TODO: Create linear layer for 2D input (frequency + intensity)
        self.linear = None
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        # TODO: Forward pass with 2D features
        return None
    
    def predict_ceremony(self, features: torch.Tensor) -> torch.Tensor:
        """Predict ceremony using both frequency and intensity."""
        with torch.no_grad():
            logits = self.forward(features)
            probabilities = F.softmax(logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1)
            return predictions, probabilities

# TRIAL: Train enhanced oracle and compare accuracy to single-feature version
# SUCCESS: Achieve >85% accuracy with combined features
print("🍜 Cook Oh-Pai-Timizer's enhancement ready - implement the TODOs!")

### Extension 2: He-Ao-World's Bell Maintenance Mishap
*"Oh dear! I was cleaning the bells and... well, some got mixed up..."*

*He-Ao-World shuffles over, looking particularly apologetic today*

*"I was polishing the ceremonial bells this morning when my old hands slipped. Some bells got moved to different positions, and I'm afraid their frequency measurements might be... inconsistent. Some readings seem to be in the wrong octave, others shifted by harmonic intervals. The bell master will be most displeased if we can't sort this out!"*

**NEW CONCEPTS:** Data preprocessing, outlier detection, robust training  
**DIFFICULTY:** +25% (still Dan 1, but messier real-world data)

In [ ]:
def create_messy_bell_data(frequencies, labels, chaos_factor: float = 0.3):
    """
    Simulate He-Ao-World's bell maintenance mishaps.
    
    Common "accidents":
    - Some frequencies doubled (wrong octave)
    - Some frequencies halved (wrong octave) 
    - Random frequency shifts (bell positioning errors)
    - Occasional completely wrong readings (sensor problems)
    
    Returns:
        Corrupted frequencies with same labels
    """
    # TODO: Implement realistic data corruption
    # Hint: Randomly select subset of data to corrupt
    # Hint: Apply different types of corruption (doubling, halving, shifting)
    pass

def preprocess_bell_data(frequencies, labels):
    """
    Clean and preprocess potentially corrupted bell data.
    
    Returns:
        Tuple of (cleaned_frequencies, cleaned_labels, outlier_mask)
    """
    # TODO: Implement data cleaning strategies
    # Hint: Detect outliers using statistical methods
    # Hint: Consider frequency ranges that make physical sense
    # Hint: Remove or correct obviously wrong measurements
    pass

def robust_training(model, features, target, **kwargs):
    """
    Train model with robustness techniques for noisy data.
    
    Returns:
        Trained model with improved noise resistance
    """
    # TODO: Implement robust training techniques
    # Hint: Consider data augmentation
    # Hint: Use validation set to monitor overfitting to noise
    pass

# TRIAL: Clean messy data and achieve comparable accuracy to clean data
# SUCCESS: Maintain >75% accuracy even with 30% corrupted data
print("🧹 He-Ao-World's challenge ready - time to clean up the mess!")

### Extension 3: Master Ao-Tougrad's Probability Mysteries
*"The wise oracle not only predicts, but reveals the uncertainty within..."*

*Master Ao-Tougrad emerges from the shadows, eyes gleaming with hidden knowledge*

*"You have learned to name the ceremonies, young seeker, but do you understand the depths of uncertainty? A true master knows not just what the bell suggests, but how confident that suggestion should be. When the bells ring at the boundaries between ceremonies, your oracle must speak of doubt as clearly as it speaks of certainty."*

**NEW CONCEPTS:** Prediction confidence, uncertainty quantification, probability interpretation  
**DIFFICULTY:** +35% (still Dan 1, but deeper understanding)

In [ ]:
def analyze_prediction_confidence(model, frequencies, labels, confidence_threshold: float = 0.7):
    """
    Analyze when the model is confident vs uncertain in its predictions.
    
    Args:
        confidence_threshold: Minimum probability to consider "confident"
    
    Returns:
        Dictionary with confidence analysis results
    """
    # TODO: Implement confidence analysis
    # Hint: Use softmax probabilities to measure confidence
    # Hint: Identify which frequency ranges cause uncertainty
    # Hint: Compare accuracy for high vs low confidence predictions
    pass

def visualize_uncertainty_landscape(model, freq_min: float = 150, freq_max: float = 500):
    """
    Create a detailed visualization of how prediction confidence varies across frequencies.
    
    Shows:
    - Predicted ceremony for each frequency
    - Confidence level (max probability)
    - Uncertainty regions (where multiple ceremonies are likely)
    """
    # TODO: Create comprehensive uncertainty visualization
    # Hint: Sample frequencies across the range
    # Hint: Show probability distribution for each frequency
    # Hint: Highlight boundary regions between ceremonies
    pass

def uncertainty_aware_predictions(model, frequencies, min_confidence: float = 0.6):
    """
    Make predictions with uncertainty awareness.
    
    Returns:
        Tuple of (predictions, confidences, uncertain_mask)
        - uncertain_mask: True where model confidence is below threshold
    """
    # TODO: Implement uncertainty-aware prediction system
    # Hint: Flag predictions with low confidence
    # Hint: Consider returning "uncertain" as a valid prediction
    pass

# TRIAL: Identify uncertainty patterns and achieve >90% accuracy on confident predictions
# SUCCESS: Demonstrate understanding of when the model should be cautious
# MASTERY: Explain why certain frequencies create uncertainty
print("🌊 Master Ao-Tougrad's mysteries await your deeper understanding...")

### Extension 4: Master Pai-Torch's Harmonic Wisdom
*"True mastery comes from understanding not just what is, but what could be..."*

*Master Pai-Torch sits in deep contemplation, then speaks with ancient wisdom*

*"You have learned to recognize the ceremonies from their frequencies, grasshopper. But the deepest wisdom lies in understanding the harmonics—the mathematical relationships between the tones. Can your oracle learn to generate new bell frequencies that would ring true for each ceremony? This is the path from recognition to creation, from understanding to wisdom."*

**NEW CONCEPTS:** Reverse engineering, feature generation, model interpretation  
**DIFFICULTY:** +45% (still Dan 1, but creative application)

In [ ]:
def reverse_engineer_ceremonies(model, target_ceremony: int, n_samples: int = 100):
    """
    Use the trained model to find frequencies that strongly indicate a specific ceremony.
    
    Args:
        target_ceremony: Which ceremony to generate frequencies for (0-3)
        n_samples: How many candidate frequencies to test
    
    Returns:
        Frequencies that the model predicts as target_ceremony with high confidence
    """
    # TODO: Implement reverse engineering
    # Hint: Test many frequencies and find ones with high target ceremony probability
    # Hint: Use model.predict_ceremony() to get probabilities
    # Hint: Return frequencies where target ceremony has >90% probability
    pass

def analyze_learned_patterns(model):
    """
    Analyze what patterns the model has learned about ceremony frequencies.
    
    Reveals:
    - Decision boundaries between ceremonies
    - Learned weight patterns
    - Frequency ranges the model associates with each ceremony
    """
    # TODO: Implement pattern analysis
    # Hint: Examine model weights to understand what it learned
    # Hint: Find decision boundaries by testing edge frequencies
    # Hint: Compare learned patterns to original ceremony definitions
    pass

def generate_synthetic_bell_ceremonies(model, ceremony_id: int, n_bells: int = 10):
    """
    Generate new bell frequencies that would be appropriate for a given ceremony.
    
    Uses the trained model's understanding to create realistic new data.
    
    Returns:
        Array of frequencies that model strongly associates with ceremony_id
    """
    # TODO: Implement synthetic data generation
    # Hint: Use optimization to find frequencies that maximize target ceremony probability
    # Hint: Start with random frequencies and adjust to increase target probability
    # Hint: Ensure generated frequencies are physically realistic (150-500 Hz)
    pass

def validate_harmonic_wisdom(model, generated_frequencies, target_ceremony):
    """
    Test whether generated frequencies actually work as intended.
    
    Returns:
        Analysis of how well the generated frequencies match the target ceremony
    """
    # TODO: Implement validation of generated frequencies
    # Hint: Test generated frequencies against the model
    # Hint: Compare to original ceremony frequency ranges
    # Hint: Check if generated frequencies cluster in expected ranges
    pass

# TRIAL: Generate 10 new frequencies for each ceremony that model classifies correctly
# SUCCESS: Generated frequencies match expected ceremony patterns
# MASTERY: Understand the relationship between model weights and ceremony characteristics
print("🎼 Master Pai-Torch's harmonic challenge - from learning to creation!")

## 🔥 CORRECTING YOUR FORM: A STANCE IMBALANCE

*Master Pai-Torch observes your training ritual with a careful eye. "Your eager mind races ahead of your disciplined form, grasshopper. See how your multi-class classification stance wavers?"*

*A previous disciple left this flawed ceremony classification ritual. Your form has become unsteady—can you restore proper technique?*

In [ ]:
def unsteady_ceremony_training(model, features, target, epochs=1000):
    """This ceremony training stance has lost its balance - your form needs correction! 🔔"""
    # Using wrong loss function for multi-class!
    criterion = nn.BCELoss()  # This is for binary classification!
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        # Forward pass
        logits = model(features)
        
        # Applying sigmoid instead of softmax for multi-class!
        predictions = torch.sigmoid(logits)
        loss = criterion(predictions, target.float())
        
        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # Wrong order! Should clear gradients BEFORE forward pass
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    return model

# DEBUGGING CHALLENGE: Can you spot the THREE critical errors in this training ritual?
# ERROR 1: Using BCELoss instead of CrossEntropyLoss for multi-class classification
# ERROR 2: Using sigmoid instead of softmax for multi-class probabilities  
# ERROR 3: Clearing gradients AFTER optimizer.step() instead of BEFORE forward pass
# 
# MASTER'S WISDOM: "Multi-class classification requires different sacred rites than binary.
#                   The CrossEntropy loss already contains the softmax wisdom within.
#                   And always banish the old gradient spirits before summoning new ones."

print("🐛 DEBUGGING TEMPLE: Find and fix the three errors in this ceremony training!")
print("     Hint 1: What loss function should you use for 4-class classification?")
print("     Hint 2: What activation should convert logits to probabilities for multiple classes?")
print("     Hint 3: When should you clear the gradient accumulation?")

## 🎭 FINAL REFLECTION: The Bell Master's Wisdom

*Master Pai-Torch rings each of the four ceremonial bells in sequence, their tones echoing through the temple halls*

"Listen, grasshopper. Do you hear how each bell speaks its truth? Your neural oracle has learned what took the ancient bell masters lifetimes to understand—that within the music of frequency lies the language of purpose."

"You have mastered:
- **Multi-class classification** - distinguishing between many possibilities, not just two
- **Softmax activation** - transforming raw scores into the language of probability
- **Cross-entropy loss** - measuring the distance between prediction and truth across multiple classes
- **Probability interpretation** - understanding confidence and uncertainty in your oracle's voice"

"Remember this lesson: classification is not just about being right or wrong. It is about understanding the spectrum of possibility, the gradations of certainty, and the wisdom to know when doubt itself is the truest answer."

*The master's eyes twinkle with pride*

"The bells will sing again tomorrow, and your oracle will be ready to guide the faithful to their proper ceremonies. This is the way of the Temple Sweeper who has begun to understand the deeper harmonies of neural wisdom."

---

**🏆 Mastery Achieved When:**
- [ ] Model achieves >80% accuracy on ceremony classification
- [ ] Understanding of softmax vs sigmoid activation
- [ ] Correct implementation of CrossEntropyLoss
- [ ] Ability to interpret prediction probabilities
- [ ] Recognition of multi-class vs binary classification differences

**🎯 Next Steps in Your Journey:**
Ready to advance? Dan 2 awaits with deeper mysteries of regularization and validation, where you'll learn to protect your models from the demons of overfitting...